In [165]:
"""for pulling down data from hr.info API on response level and sending to CSV"""
#locations: https://api.hpc.tools/v1/public/location
#clusters: https://api.hpc.tools/v1/public/global-cluster ["code":"SHL"]
#https://api.hpc.tools/docs/v1/
#boundary cat by looping through locations

import urllib.request, json
import csv
import itertools
import math
import collections
import sys

import grequests
import pandas as pd
from pandas.io.json import json_normalize

sys.path.insert(0, '../regpy')
import sc_pull

%load_ext autoreload
%autoreload 1
pd.set_option('display.max_columns', 500)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
with urllib.request.urlopen("https://api.hpc.tools/v1/public/fts/flow?globalClusterCode=shl") as url:
    data = json.loads(url.read().decode())

ret = json_normalize(data)

#total number of flows: 9,294
ret['data.incoming.flowCount']

#total amount of incoming funding: 7,857,348,149
ret['data.incoming.fundingTotal']

#pledged: 37,481,772
ret['data.incoming.fundingTotal']

In [389]:
import imp
imp.reload(sc_pull)

loaded 227 SC entries


<module 'sc_pull' from '../regpy/sc_pull.py'>

In [ ]:
#outside
# j = j.merge(hist, left_on = 'fts.url', right_on = 'sc.fts_url')

In [390]:
class fts(object):

    def __init__(self, test = None):
        self.fts = self.master_pull(test)
        self.merge_sc()
        
        self.flows = self.extract_flows()
        
        #remove flows col as it's quite large
        self.fts = self.fts.drop('fts.data.flows', axis = 1)
        
    def extract_flows(self):
        """take all flows and add to df"""
        
        flow_df = pd.DataFrame()
        for v in self.fts.iterrows():
            norm = json_normalize(v[1]['fts.data.flows'])
            norm['sc.uid'] = v[1]['sc.uid']

            if len(flow_df) == 0:
                flow_df = norm
            else:
                #need to make sure we're adding t3h same c0lz
                assert(norm.columns.all(flow_df.columns))
                flow_df = flow_df.append(norm, ignore_index = True)
        
        return flow_df
    
    def master_pull(self, test):
        """group by year, country and pull down total values with optional filter by cluster"""
        countries = json_normalize(json.loads(urllib.request.urlopen("https://api.hpc.tools/v1/public/location")
                                              .read().decode()))['data']

        print('pulled countries')

        self.hist = sc_pull.pull()

        self.hist['sc.fts_url'] = self.hist.apply(lambda x : 'https://api.hpc.tools/v1/public/fts/flow?countryISO3={0}&year={1}&globalClusterCode=shl'
                                     .format(x['sc.iso3'], x['sc.year']), axis = 1)
        
        if test:
            hrefz = self.hist['sc.fts_url'][:test]
        else:
            hrefz = self.hist['sc.fts_url']

        def exception_handler(request, exception):
             print('Bad URL for ' + request)

        resps = []
        rs = (grequests.get(ref) for ref in hrefz)
        resps += grequests.map(rs, exception_handler = exception_handler, size = 25)

        good_resps = []
        bad_resps = []
        for r in resps:
            load = json.loads(r.content)
            load['url'] = r.url
            if r.status_code == 200:
                good_resps.append(load)
            else:
                bad_resps.append(load)
        
        print('num bad resps: ' + str(len(bad_resps)))
        return json_normalize(good_resps).add_prefix('fts.')
    
    def merge_sc(self):
        self.fts = self.fts.merge(self.hist, left_on = 'fts.url', right_on = 'sc.fts_url')
        
        

In [391]:
r = fts(test = 92)

pulled countries
loaded 227 SC entries
num bad resps: 0


/Users/ewanog/Documents/work/code/venv/p3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [393]:
ft = r.fts

In [394]:
ft.head(5)

,fts.data.incoming.flowCount,fts.data.incoming.fundingTotal,fts.data.incoming.pledgeTotal,fts.data.internal.flowCount,fts.data.outgoing.flowCount,fts.data.outgoing.fundingTotal,fts.data.outgoing.pledgeTotal,fts.meta.count,fts.meta.language,fts.status,fts.url,sc.region,sc.region_url,sc.country,sc.country_url,sc.year,sc.response,sc.lead_agency,sc.disaster_type,sc.website_url,sc.colead_agency,sc.activation_date,sc.deactivation_date,sc.num_clust,sc.iso3,sc.uid,sc.fts_url
0,152,113366675,1851852,0,0,0,0,152,en,ok,https://api.hpc.tools/v1/public/fts/flow?count...,Asia-Pacific,http://sheltercluster.org/asiapacific,Pakistan,http://sheltercluster.org/asiapacific/pakistan,2005,Pakistan - Kashmir Earthquake,IOM,Natural Disaster,NaN,NaN,NaN,NaN,1,PAK,PAK2005,https://api.hpc.tools/v1/public/fts/flow?count...
1,22,8429530,46154,0,0,0,0,22,en,ok,https://api.hpc.tools/v1/public/fts/flow?count...,Africa,http://sheltercluster.org/africa,Democratic Republic of Congo,http://sheltercluster.org/africa/democratic-re...,2006,Democratic Republic of Congo - Conflict,UNICEF,Conflict,http://sheltercluster.org/response/democratic-...,CRS,NaN,NaN,1,COD,COD2006,https://api.hpc.tools/v1/public/fts/flow?count...
2,55,32278571,554785,0,0,0,0,55,en,ok,https://api.hpc.tools/v1/public/fts/flow?count...,Asia-Pacific,http://sheltercluster.org/asiapacific,Indonesia,http://sheltercluster.org/asiapacific/indonesia,2006,Indonesia - Yogyakarta Earthquake,IFRC,Natural Disaster,http://sheltercluster.org/response/yogyakarta-...,NaN,NaN,NaN,1,IDN,IDN2006,https://api.hpc.tools/v1/public/fts/flow?count...
3,71,85196751,0,0,0,0,0,71,en,ok,https://api.hpc.tools/v1/public/fts/flow?count...,MENA,http://sheltercluster.org/mena,Lebanon,NaN,2006,Lebanon - Conflict,UNHCR,Conflict,NaN,NaN,NaN,NaN,1,LBN,LBN2006,https://api.hpc.tools/v1/public/fts/flow?count...
4,1,341267,0,0,0,0,0,1,en,ok,https://api.hpc.tools/v1/public/fts/flow?count...,Africa,http://sheltercluster.org/africa,Liberia,NaN,2006,Liberia - Conflict,UNHCR,Conflict,NaN,NaN,NaN,NaN,1,LBR,LBR2006,https://api.hpc.tools/v1/public/fts/flow?count...


In [395]:
def pop_flows():
    g = fs.groupby(by = ['sc.uid', 'boundary', 'status']).agg({'amountUSD' : 'sum'})
    g = g.reset_index()

    ft['incoming_commitment'] = None
    ft['incoming_paid'] = None
    ft['incoming_pledge'] = None
    ft['outgoing_commitment'] = None
    ft['outgoing_paid'] = None
    ft['outgoing_pledge'] = None
    ft['internal_commitment'] = None
    ft['internal_paid'] = None
    ft['internal_pledge'] = None

    for v in g.iterrows():
        var_nm = v[1]['boundary'] + '_' + v[1]['status']
        if len(ft.loc[ft['sc.uid'] == v[1]['sc.uid']]) != 1:
            print(v[1]['sc.uid'])
        ft.loc[ft['sc.uid'] == v[1]['sc.uid'], var_nm] = v[1]['amountUSD']

pop_flows()

In [346]:
g = fs.groupby(by = ['sc.uid', 'boundary', 'status']).agg({'amountUSD' : 'sum'})

g.reset_index()

ret = collections.OrderedDict({
        'inc_commitment' : None,
        'inc_paid' : None,
        'inc_pledge' : None,
        'out_commitment' : None,
        'out_paid' : None,
        'out_pledge' : None
        })

#add cols to fs
for v in ret.keys():
    ft[v] = None
    


In [47]:
def cost_bdown(v):
    """breakdown cost by commitment, paid and pledge.
        create cols:
            inc_commitment
            inc_paid
            inc_pledge
            out_commitment
            out_paid
            out_pledge
    """
    ret = collections.OrderedDict({
            'inc_commitment' : None,
            'inc_paid' : None,
            'inc_pledge' : None,
            'out_commitment' : None,
            'out_paid' : None,
            'out_pledge' : None
            })
    
    #add cols to fs
    for v in ret.keys():
        fs[v] = None
    
    if len(v['fts.data.flows']) > 0:
        gpd = json_normalize(v['fts.data.flows'][0]).groupby(by = 'status')['amountUSD'].sum()
    
        for k in gpd.keys():
            if k in ret:
                ret[k] = gpd[k]
    
    print(pd.Series(ret))
    return ret
            

def p(v):
    print(v['fts.data.flows'])
        
# j.apply(lambda x : cost_bdown(x), axis = 1)
j[['amt_commit', 'amt_paid', 'amt_pledge']] = j.apply(lambda x: pd.Series(cost_bdown(x)), axis = 1)

commitment    195340.0
paid               NaN
pledge             NaN
dtype: float64
commitment    195340.0
paid               NaN
pledge             NaN
dtype: float64
commitment    123931.0
paid               NaN
pledge             NaN
dtype: float64
commitment    2720348.0
paid                NaN
pledge              NaN
dtype: float64
commitment         NaN
paid          107789.0
pledge             NaN
dtype: float64
commitment    341267.0
paid               NaN
pledge             NaN
dtype: float64
commitment        NaN
paid          62308.0
pledge            NaN
dtype: float64
commitment    147815.0
paid               NaN
pledge             NaN
dtype: float64
commitment    115976.0
paid               NaN
pledge             NaN
dtype: float64
commitment    39388.0
paid              NaN
pledge            NaN
dtype: float64
commitment        NaN
paid          93967.0
pledge            NaN
dtype: float64
commitment    341253.0
paid               NaN
pledge             NaN
dtype: float6

commitment    30211.0
paid              NaN
pledge            NaN
dtype: float64
commitment          NaN
paid          3566224.0
pledge              NaN
dtype: float64
commitment    230800.0
paid               NaN
pledge             NaN
dtype: float64
commitment    200000.0
paid               NaN
pledge             NaN
dtype: float64
commitment    536979.0
paid               NaN
pledge             NaN
dtype: float64
commitment    536979.0
paid               NaN
pledge             NaN
dtype: float64
commitment    271370.0
paid               NaN
pledge             NaN
dtype: float64
commitment          NaN
paid          1770380.0
pledge              NaN
dtype: float64
commitment          NaN
paid          2700000.0
pledge              NaN
dtype: float64
commitment    7600000.0
paid                NaN
pledge              NaN
dtype: float64
commitment         NaN
paid          227530.0
pledge             NaN
dtype: float64
commitment    19318.0
paid              NaN
pledge            NaN
d

In [243]:
bounds = {}
def get_bound_types(x):
    """see what count boundary transactions we have
        {'incoming': 4889, 'internal': 10, 'outgoing': 23}
    """
    norm = json_normalize(x['fts.data.flows'])
    
    if 'boundary' in norm:
        for v in norm['boundary']:
            if v == 'outgoing':
                print(x)
                raise('sfd')

            if v not in bounds:
                bounds[v] = 1
            else:
                bounds[v] += 1
    
j.apply(lambda x : get_bound_types(x), axis = 1)

fts.data.flows                    [{'id': '43607', 'versionId': 1, 'description'...
fts.data.incoming.flowCount                                                       8
fts.data.incoming.fundingTotal                                              1597544
fts.data.incoming.pledgeTotal                                                     0
fts.data.internal.flowCount                                                       0
fts.data.outgoing.flowCount                                                       1
fts.data.outgoing.fundingTotal                                               150000
fts.data.outgoing.pledgeTotal                                                     0
fts.meta.count                                                                    9
fts.meta.language                                                                en
fts.status                                                                       ok
fts.url                           https://api.hpc.tools/v1/public/fts/flow?c

TypeError: ('exceptions must derive from BaseException', 'occurred at index 9')

In [102]:
#test to see if we're figuring out outgoing correctly
ent = j[j['fts.data.incoming.fundingTotal'] == 1597544]

#regular:
json_normalize(ent['fts.data.flows'][9]).groupby(by = 'status')['amountUSD'].sum()
# commitment    1130923
# paid           616621

#subtract:
r = json_normalize(ent['fts.data.flows'][9]).groupby(by = ['status', 'boundary'])['amountUSD'].sum()

list(set(r.index.get_level_values('status')))

r.keys()

KeyError: 'status'